In [1]:
# if open from google colab, run this command
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/sales-prediction-gru/ipynb

Mounted at /content/drive
/content/drive/MyDrive/Github/sales-prediction-gru/ipynb


In [2]:
# import numpy as np
# import pandas as pd 
# import matplotlib.pyplot as plt
# import os
# from keras.layers import LSTM,Dense,Dropout
# from tensorflow.keras import optimizers, Sequential, Model, callbacks
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler, StandardScaler

import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers, Sequential, Model, callbacks

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [3]:
df = pd.read_csv('../data/fakta_penjualan_unv_202112091006.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning:

Columns (0,10,12) have mixed types.Specify dtype option on import or set low_memory=False.



In [4]:
df = df.drop(['nonota', 'namabarang', 'satuan', 'hargajual', 'hargabeli', 'hargajual2', 'kodejenis', 'jenis', 'keuntungan', 'kodecabang', 'cabang'], axis=1)

In [5]:
df['barcode'] = df['barcode'].astype(str).str[:13]
df['barcode'] = df['barcode'].astype(int)
df['qty'] = df['qty'].astype(int)

In [6]:
test = df[df['tahun']==2019]

years = [2016, 2017, 2018]
train = df[df.tahun.isin(years)]

In [7]:
def column_to_date(data):
    # create date column from hari, bulan, tahun column
    cols = ["hari","bulan","tahun"]
    data['date'] = data[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
    
    # drop hari, bulan, tahun column
    data = data.drop(['hari', 'bulan', 'tahun'], axis=1)
    
    # transformn date column to datetime
    data['date'] = pd.to_datetime(data['date'])
    
    return data

In [8]:
test = column_to_date(test)
train = column_to_date(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
train['month_yr'] = train['date'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [10]:
train['month_yr'] = train['month_yr'].map({'01-16':0,'02-16':1,'03-16':2,'04-16':3,'05-16':4,'06-16':5,'07-16':6,'08-16':7,'09-16':8,'10-16':9,'11-16':10,'12-16':11,
                                          '01-17':12,'02-17':13,'03-17':14,'04-17':15,'05-17':16,'06-17':17,'07-17':18,'08-17':19,'09-17':20,'10-17':21,'11-17':22,'12-17':23,
                                          '01-18':24,'02-18':25,'03-18':26,'04-18':27,'05-18':28,'06-18':29,'07-18':30,'08-18':31,'09-18':32,'10-18':33,'11-18':34,'12-18':35})

In [11]:
train

,qty,barcode,date,month_yr
65817,3,8999999036607,2016-01-01,0
65818,1,8999999003722,2016-01-01,0
65819,1,8999999036898,2016-01-01,0
66058,3,8992747180201,2016-01-01,0
66059,2,8999999049034,2016-01-01,0
...,...,...,...,...
1096033,1,8999999033224,2018-12-31,35
1096034,1,8999999528805,2018-12-31,35
1096035,1,8999999390198,2018-12-31,35
1096036,1,8999999526870,2018-12-31,35


In [12]:
dataset = train.pivot_table(index=['barcode'], values=['qty'], columns=['month_yr'], fill_value=0, aggfunc='sum')

In [13]:
dataset.reset_index(inplace = True)

In [14]:
dataset.head()

barcode qty                                  ...              \
month_yr                  0   1   2   3   4   5   6   7   8  ...  26  27  28   
0         4800888137319  15  23  23  12  23  17  22  14   9  ...  16  15  15   
1         4800888141101  46  41  42  48  34  32  45  42  40  ...   0   0   0   
2         4800888141125  75  89  77  87  87  76  72  76  49  ...  71  66  71   
3         4800888157942  23  21  29  32  19  26  19  24  12  ...   1   2   2   
4         4800888157959  11  13  12  13   9   5   5   4   7  ...   8  11   4   

                                      
month_yr  29  30  31  32  33  34  35  
0          9  17  11  13  14  14  13  
1          0   0   0   0   0   0   0  
2         56  63  35  13  41  54  59  
3          0   1   5   4   1   1   4  
4         10   5   6   6   3   3   7  

[5 rows x 37 columns]

In [15]:
test_des = test[pd.to_datetime(test['date']).dt.month == 12]

In [16]:
test_des = test_des.drop_duplicates(subset=['barcode'])

In [17]:
test_des = test_des.drop(['qty', 'date'], axis=1)

In [18]:
test_clean = test.drop(['qty', 'date'], axis=1)

In [19]:
test_clean = test_clean.drop_duplicates(subset=['barcode'])

In [20]:
print(dataset.shape)
print(test.shape)
print(test_des.shape)
print(test_clean.shape)

(1368, 37)
(350956, 3)
(973, 1)
(1038, 1)


In [21]:
# Now we will merge our pivot table with the test because we want to keep the data of items we have predict
dataset = pd.merge(test_clean, dataset, on=['barcode'], how='left')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:4150: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [22]:
# lets fill all NaN values with 0
dataset.fillna(0,inplace = True)

In [23]:
dataset.shape

(1038, 37)

In [24]:
# we will drop barcode because we do not need them
# we are teaching our model how to generate the next sequence 
dataset.drop(['barcode'], inplace=True, axis=1)
dataset.head()

,"(qty, 0)","(qty, 1)","(qty, 2)","(qty, 3)","(qty, 4)","(qty, 5)","(qty, 6)","(qty, 7)","(qty, 8)","(qty, 9)",...,"(qty, 26)","(qty, 27)","(qty, 28)","(qty, 29)","(qty, 30)","(qty, 31)","(qty, 32)","(qty, 33)","(qty, 34)","(qty, 35)"
0,134.0,120.0,117.0,116.0,143.0,141.0,86.0,99.0,107.0,101.0,...,146.0,155.0,96.0,131.0,68.0,108.0,75.0,107.0,52.0,126.0
1,47.0,44.0,42.0,45.0,28.0,52.0,26.0,29.0,37.0,269.0,...,330.0,370.0,351.0,437.0,323.0,272.0,271.0,398.0,321.0,327.0
2,49.0,55.0,39.0,51.0,57.0,35.0,47.0,15.0,16.0,39.0,...,47.0,60.0,35.0,41.0,68.0,53.0,67.0,79.0,46.0,43.0
3,57.0,57.0,53.0,44.0,50.0,48.0,53.0,44.0,39.0,52.0,...,34.0,39.0,17.0,40.0,39.0,40.0,44.0,19.0,10.0,33.0
4,189.0,168.0,177.0,113.0,153.0,161.0,142.0,150.0,120.0,135.0,...,250.0,204.0,201.0,206.0,212.0,230.0,200.0,214.0,202.0,225.0


In [25]:
# X we will keep all columns execpt the last one 
X_train = np.expand_dims(dataset.values[:,:-1], axis = 2)
# the last column is our label
y_train = dataset.values[:,-1:]

# for test we keep all the columns execpt the first one
X_test = np.expand_dims(dataset.values[:,1:], axis = 2)
y_test = dataset.values[:,:1]

# lets have a look on the shape 
print(X_train.shape, y_train.shape, X_test.shape)

(1038, 35, 1) (1038, 1) (1038, 35, 1)


In [26]:
# our defining our model 
# my_model = Sequential()
# my_model.add(L.GRU(units=64, input_shape=(35,1)))
# my_model.add(L.Dropout(0.4))
# my_model.add(L.Dense(1))
# my_model.compile(loss='mse', optimizer='adam')
# my_model.summary()

serie_size =  X_train.shape[1] # 12
n_features =  X_train.shape[2] # 1

epochs = 1000
batch = 4096
lr = 0.0001

gru_model = Sequential()
gru_model.add(L.GRU(10, input_shape=(serie_size, n_features), return_sequences=True))
# gru_model.add(L.GRU(6, activation='relu', return_sequences=True))
gru_model.add(L.GRU(1, activation='relu'))
gru_model.add(L.Dense(10, kernel_initializer='glorot_normal', activation='relu'))
# gru_model.add(L.Dense(10, kernel_initializer='glorot_normal', activation='relu'))
gru_model.add(L.Dense(1))
gru_model.summary()

adam = optimizers.Adam(lr)
gru_model.compile(loss='mse', optimizer=adam)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 35, 10)            390       
                                                                 
 gru_1 (GRU)                 (None, 1)                 39        
                                                                 
 dense (Dense)               (None, 10)                20        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 460
Trainable params: 460
Non-trainable params: 0
_________________________________________________________________


In [27]:
# early_stop = callbacks.EarlyStopping(patience = 5)
# my_model.fit(X_train, y_train, batch_size=64, epochs=100, verbose=2)

early_stop = callbacks.EarlyStopping(patience = 5)
gru_history = gru_model.fit(X_train, y_train,
                              validation_data=(X_test, y_test), 
                              batch_size=batch, 
                              epochs=epochs, 
                              verbose=2,
                              callbacks=[early_stop])

Epoch 1/1000
1/1 - 4s - loss: 5129.4355 - val_loss: 6706.3091 - 4s/epoch - 4s/step
Epoch 2/1000
1/1 - 0s - loss: 5129.4297 - val_loss: 6706.3037 - 84ms/epoch - 84ms/step
Epoch 3/1000
1/1 - 0s - loss: 5129.4229 - val_loss: 6706.2979 - 85ms/epoch - 85ms/step
Epoch 4/1000
1/1 - 0s - loss: 5129.4160 - val_loss: 6706.2910 - 80ms/epoch - 80ms/step
Epoch 5/1000
1/1 - 0s - loss: 5129.4097 - val_loss: 6706.2852 - 85ms/epoch - 85ms/step
Epoch 6/1000
1/1 - 0s - loss: 5129.4033 - val_loss: 6706.2778 - 79ms/epoch - 79ms/step
Epoch 7/1000
1/1 - 0s - loss: 5129.3960 - val_loss: 6706.2725 - 81ms/epoch - 81ms/step
Epoch 8/1000
1/1 - 0s - loss: 5129.3896 - val_loss: 6706.2671 - 80ms/epoch - 80ms/step
Epoch 9/1000
1/1 - 0s - loss: 5129.3838 - val_loss: 6706.2603 - 93ms/epoch - 93ms/step
Epoch 10/1000
1/1 - 0s - loss: 5129.3770 - val_loss: 6706.2544 - 86ms/epoch - 86ms/step
Epoch 11/1000
1/1 - 0s - loss: 5129.3701 - val_loss: 6706.2485 - 78ms/epoch - 78ms/step
Epoch 12/1000
1/1 - 0s - loss: 5129.3638 - va

In [28]:
# fig, ax1 = plt.subplots(1, sharey=True,figsize=(22,7))

# ax1.plot(my_model.history['loss'], label='Train loss')
# ax1.plot(my_model.history['mean_squared_error'], label='mse')
# ax1.legend(loc='best')
# ax1.set_title('GRU')
# ax1.set_xlabel('Epochs')
# ax1.set_ylabel('MSE')

# plt.show()

In [29]:
lstm_train_pred = gru_model.predict(X_train)
# lstm_val_pred = my_model.predict(X_test)
print('Train rmse:', np.sqrt(mean_squared_error(y_train, lstm_train_pred)))
# print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, lstm_val_pred)))

Train rmse: 71.57456465912476


In [30]:
# # creating submission file 
# submission_pfs = my_model.predict(X_test)
# # we will keep every value between 0 and 20
# submission_pfs = submission_pfs.clip(0,20)
# # creating dataframe with required columns 
# submission = pd.DataFrame({'barcode':test['barcode'], 'qty_month':submission_pfs.ravel()})
# # creating csv file from dataframe
# submission.to_csv('sub_pfs.csv', index=False)